In [3]:
import pandas as pd
analysis_df = pd.read_csv('final_analysis (1).csv')
len(analysis_df)

1235

In [4]:
analysis_df

,Hash,Author,Message,Filename,Change Type,Source Code (before),Source Code (current),Diff,LLM_inference,Overall_diff,Rectified Message
0,4b97a810b509c93f44be4c037c7aa18fb8922884,Daniel Han,"Pre-release 2023 December version (Mistral, Pr...",README.md,MODIFY,"<div class=""align-center"">\n <img src=""./imag...","<div class=""align-center"">\n <img src=""./imag...","@@ -1,23 +1,25 @@\n <div class=""align-center"">...",add more info about nvidia gpus,"@@ -1,23 +1,25 @@\n <div class=""align-center"">...",add more info about nvidia gpus
1,4b97a810b509c93f44be4c037c7aa18fb8922884,Daniel Han,"Pre-release 2023 December version (Mistral, Pr...",Discord.png,MODIFY,PNG\r\n\n,PNG\r\n\n,Binary files a/images/Discord.png and b/images...,distro binary files,"@@ -1,23 +1,25 @@\n <div class=""align-center"">...",add more info about nvidia gpus
2,4b97a810b509c93f44be4c037c7aa18fb8922884,Daniel Han,"Pre-release 2023 December version (Mistral, Pr...",LAION 2GPU.png,ADD,NaN,PNG\r\n\n,Binary files /dev/null and b/images/LAION 2GPU...,distinguish 2gpu image from null,"@@ -1,23 +1,25 @@\n <div class=""align-center"">...",add more info about nvidia gpus
3,4b97a810b509c93f44be4c037c7aa18fb8922884,Daniel Han,"Pre-release 2023 December version (Mistral, Pr...",LAION 2GPU.svg,DELETE,"<?xml version=""1.0"" encoding=""utf-8"" standalon...",NaN,"@@ -1,1518 +0,0 @@\n-<?xml version=""1.0"" encod...",add missing missing nodes in skeleton skeleton,"@@ -1,23 +1,25 @@\n <div class=""align-center"">...",add more info about nvidia gpus
4,4b97a810b509c93f44be4c037c7aa18fb8922884,Daniel Han,"Pre-release 2023 December version (Mistral, Pr...",SlimOrca 1GPU.svg,DELETE,"<?xml version=""1.0"" encoding=""utf-8"" standalon...",NaN,"@@ -1,1424 +0,0 @@\n-<?xml version=""1.0"" encod...",add missing missing svg elements,"@@ -1,23 +1,25 @@\n <div class=""align-center"">...",add more info about nvidia gpus
...,...,...,...,...,...,...,...,...,...,...,...
1230,9050d6f5278e2373e66d17fc9ac8c2ec4f986e7e,Roland Tannous,tests for mxfp4 and quantized models merge fix...,run_test.sh,ADD,NaN,"#!/bin/bash\nset -e\n\necho ""=================...","@@ -0,0 +1,18 @@\n+#!/bin/bash\n+set -e\n+\n+e...",add missing line,"@@ -0,0 +1,18 @@\n+#!/bin/bash\n+set -e\n+\n+e...",add test for merged model
1231,9050d6f5278e2373e66d17fc9ac8c2ec4f986e7e,Roland Tannous,tests for mxfp4 and quantized models merge fix...,test_merged_model.py,ADD,NaN,# inference_on_merged.py\nfrom unsloth import ...,"@@ -0,0 +1,55 @@\n+# inference_on_merged.py\n+...",add test for merge,"@@ -0,0 +1,18 @@\n+#!/bin/bash\n+set -e\n+\n+e...",add test for merged model
1232,9050d6f5278e2373e66d17fc9ac8c2ec4f986e7e,Roland Tannous,tests for mxfp4 and quantized models merge fix...,train_and_merge.py,ADD,NaN,# train_and_merge.py\nfrom unsloth import Fast...,"@@ -0,0 +1,71 @@\n+# train_and_merge.py\n+from...",add examples to train_and_merge,"@@ -0,0 +1,18 @@\n+#!/bin/bash\n+set -e\n+\n+e...",add test for merged model
1233,9050d6f5278e2373e66d17fc9ac8c2ec4f986e7e,Roland Tannous,tests for mxfp4 and quantized models merge fix...,test_merge_4bit_validation.py,ADD,NaN,from unsloth import FastLanguageModel\nfrom un...,"@@ -0,0 +1,223 @@\n+from unsloth import FastLa...",add test case for formatting prompts,"@@ -0,0 +1,18 @@\n+#!/bin/bash\n+set -e\n+\n+e...",add test for merged model


In [14]:
import pandas as pd
radon_df = pd.read_csv('radon_results.csv')

In [23]:
print(req_df.iloc[1]['Diff'])

@@ -19,6 +19,14 @@ import warnings
 import gc
 warnings.filterwarnings(action = "ignore", category = UserWarning, module = "torch")
 import bitsandbytes as bnb
+from transformers.models.llama.modeling_llama import logger
+
+__version__ = "2023.12"
+__all__ = [
+    "prepare_model_for_kbit_training",
+    "patch_tokenizer",
+    "print_unsloth_message",
+]
 
 
 def prepare_model_for_kbit_training(
@@ -59,3 +67,38 @@ def prepare_model_for_kbit_training(
 
     return model
 pass
+
+
+def patch_tokenizer(model, tokenizer):
+    if not hasattr(tokenizer, "pad_token") or tokenizer.pad_token is None:
+        # Fixes https://github.com/unslothai/unsloth/issues/5
+        if hasattr(tokenizer, "unk_token"):
+            tokenizer.add_special_tokens({"pad_token" : tokenizer.unk_token})
+            tokenizer.pad_token = tokenizer.unk_token
+        else:
+            logger.warning_one(
+                f"{model.config._name_or_path} does not have a padding or unknown token!\n"\
+             

In [ ]:
import pandas as pd
pd.read_csv('')

In [25]:
print(req_df.iloc[1])

Hash                              2d5d88487463e76f75002be3b2704267ec96e68a
Author                                                     Daniel Han-Chen
Message                                                  tokenizer pad fix
Filename                                                         _utils.py
Change Type                                                         MODIFY
Source Code (before)     # Copyright 2023-present Daniel Han-Chen & the...
Source Code (current)    # Copyright 2023-present Daniel Han-Chen & the...
Diff                     @@ -19,6 +19,14 @@ import warnings\n import gc...
LLM_inference                                          add fix for unsloth
Overall_diff             @@ -19,6 +19,14 @@ import warnings\n import gc...
Rectified Message                                      add fix for unsloth
MI_Before                                                            100.0
MI_After                                                         78.270117
CC_Before                

In [35]:
print(req_df.iloc[0]['Diff'])

@@ -0,0 +1,33 @@
+# Copyright 2023-present Daniel Han-Chen & the Unsloth team. All rights reserved.
+#
+# Licensed under the Apache License, Version 2.0 (the "License");
+# you may not use this file except in compliance with the License.
+# You may obtain a copy of the License at
+#
+#     http://www.apache.org/licenses/LICENSE-2.0
+#
+# Unless required by applicable law or agreed to in writing, software
+# distributed under the License is distributed on an "AS IS" BASIS,
+# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
+# See the License for the specific language governing permissions and
+# limitations under the License.
+
+import time
+from contextlib import contextmanager
+
+
+@contextmanager
+def timer(name):
+    start = time.time()
+    yield
+    end = time.time()
+    print(f"{name} took {end - start:.2f} seconds")
+
+
+@contextmanager
+def header_footer_context(title: str, char="-"):
+    print()
+    print(f"{char}" * 50 + f" {title} " + f"{char}" 

In [ ]:
from radon.raw import analyze
raw_metrics = analyze("""def _split_tokens(tokens, token, value):
    '''Split a list of tokens on the specified token pair (token, value),
    where *token* is the token type (i.e. its code) and *value* its actual
    value in the code.
    '''
    res = [[]]
    for token_values in tokens:
        if (token, value) == token_values[:2]:
            res.append([])
            continue
        res[-1].append(token_values)
    return res
""")
print(raw_metrics.loc)

12


In [48]:
from radon.complexity import cc_visit, cc_rank
from radon.metrics import mi_visit, mi_rank

code = '''
class A(object):
    def meth(self):
        return sum(i for i in range(10) if i - 2 < 5)

def fib(n):
    if n < 2: return 1
    return fib(n - 1) + fib(n - 2)
'''
cc_blocks = cc_visit(code)
mi_score = mi_visit(code, True)  # True = return float instead of string
cc_score = sum(block.complexity for block in cc_blocks)/len(cc_blocks)
print("CC score:", cc_score)
print("CC Rank",cc_rank(cc_score))
print("MI score:", mi_score)
print("MI Rank:", mi_rank(mi_score))

CC score: 3.0
CC Rank A
MI score: 68.2206039600699
MI Rank: A


In [ ]:
df_processed["Semantic_Similarity"] = df_processed.apply(
    lambda row: semantic_similarity(str(row["Source Code (before)"]),str(row["Source Code (current)"])), axis=1
)
df_processed["Token_Similarity"] = df_processed.apply(
    lambda row: token_similarity_bleu(str(row["Source Code (before)"]), str(row["Source Code (current)"])), axis=1
)
def classify_semantic(sim):
    return "Minor" if sim >= 0.80 else "Major"

def classify_token(sim):
    return "Minor" if sim >= 0.75 else "Major"
df_processed["Semantic_Class"] = df["Semantic_Similarity"].apply(classify_semantic)
df_processed["Token_Class"] = df["Token_Similarity"].apply(classify_token)
df_processed["Classes_Agree"] = df_processed.apply(
    lambda row: "YES" if row["Semantic_Class"] == row["Token_Class"] else "NO", axis=1
)

NameError: name 'df_processed' is not defined

In [44]:
cc_blocks

[Function(name='fib', lineno=6, col_offset=0, endline=8, is_method=False, classname=None, closures=[], complexity=2),
 Class(name='A', lineno=2, col_offset=0, endline=4, methods=[Function(name='meth', lineno=3, col_offset=4, endline=4, is_method=True, classname='A', closures=[], complexity=3)], inner_classes=[], real_complexity=4),
 Function(name='meth', lineno=3, col_offset=4, endline=4, is_method=True, classname='A', closures=[], complexity=3)]

In [ ]:
for block in cc_blocks:
    print(block)

F 6:0->8 fib - 2
C 2:0->4 A - 4
M 3:4->4 A.meth - 3


In [ ]:

cc_score

9

In [37]:
req_df = radon_df[(radon_df['CC_Change']!=0) & (radon_df['LOC_Change'].notna()) & (radon_df['LOC_After']<=40) & (radon_df['LOC_Before']<=30) ]
print(req_df.iloc[0]['CC_Change'])
print(req_df.iloc[0]['Source Code (before)'])
print(req_df.iloc[0]['Source Code (current)'])
req_df

2.0
nan
# Copyright 2023-present Daniel Han-Chen & the Unsloth team. All rights reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import time
from contextlib import contextmanager


@contextmanager
def timer(name):
    start = time.time()
    yield
    end = time.time()
    print(f"{name} took {end - start:.2f} seconds")


@contextmanager
def header_footer_context(title: str, char="-"):
    print()
    print(f"{char}" * 50 + f" {title} " + f"{char}" * 50)
    yield
    print(f"{char}" * (

,Hash,Author,Message,Filename,Change Type,Source Code (before),Source Code (current),Diff,LLM_inference,Overall_diff,Rectified Message,MI_Before,MI_After,CC_Before,CC_After,LOC_Before,LOC_After,MI_Change,CC_Change,LOC_Change
923,05baf282c780961176987eb907d4d9dd7f95fbf0,Daniel Han,Fix Transformers 4.45 (#2151)\n\n* Update pypr...,__init__.py,ADD,NaN,# Copyright 2023-present Daniel Han-Chen & the...,"@@ -0,0 +1,33 @@\n+# Copyright 2023-present Da...",add contextmanager_context.py,"@@ -29,7 +29,7 @@ version = {attr = ""unsloth.m...",add docs for unsloth qlora and merge tests,100.0,87.587742,0.0,2.0,1.0,33.0,-12.412258,2.0,32.0


In [4]:
import ast
import astpretty

code = """
def example(x):
    if x > 0:
        print("Positive")
    else:
        if x == 0:
            print("Zero")
        else:
            print("Negative")
"""

tree = ast.parse(code)
astpretty.pprint(tree)

Module(
    body=[
        FunctionDef(
            lineno=2,
            col_offset=0,
            end_lineno=9,
            end_col_offset=29,
            name='example',
            args=arguments(
                posonlyargs=[],
                args=[arg(lineno=2, col_offset=12, end_lineno=2, end_col_offset=13, arg='x', annotation=None, type_comment=None)],
                vararg=None,
                kwonlyargs=[],
                kw_defaults=[],
                kwarg=None,
                defaults=[],
            ),
            body=[
                If(
                    lineno=3,
                    col_offset=4,
                    end_lineno=9,
                    end_col_offset=29,
                    test=Compare(
                        lineno=3,
                        col_offset=7,
                        end_lineno=3,
                        end_col_offset=12,
                        left=Name(lineno=3, col_offset=7, end_lineno=3, end_col_offset=8, id='x', ctx=Load())

- To calculate cyclomatic complexity(CC) radon traverses AST(Abstract Syntax Tree) and calculates number of decision blocks
- single comments -> the line contain only that comment
- docstring isn't counted in multi, its in SLOC and LLOC
The following are the definitions employed by Radon:

- LOC: The total number of lines of code. It does not necessarily correspond to the number of lines in the file.
- LLOC: The number of logical lines of code. Every logical line of code contains exactly one statement.
- SLOC: The number of source lines of code - not necessarily corresponding to the LLOC.
- Comments: The number of comment lines. Multi-line strings are not counted as comment since, to the Python interpreter, they are just strings.
- Multi: The number of lines which represent multi-line strings.
- Blanks: The number of blank lines (or whitespace-only ones).

In [3]:
!pip install astpretty

In [7]:
import pandas as pd
analysis_df = pd.read_csv('Lab_2_analysis.csv')
final_df = analysis_df[['Hash','Message','Source Code (before)','Source Code (current)','Diff','LLM_inference','New Rectified Message']]
final_df = final_df.rename(columns={'New Rectified Message':'Rectified Message'})
final_df.head()

,Hash,Message,Source Code (before),Source Code (current),Diff,LLM_inference,Rectified Message
0,4b97a810b509c93f44be4c037c7aa18fb8922884,"Pre-release 2023 December version (Mistral, Pr...","<div class=""align-center"">\n <img src=""./imag...","<div class=""align-center"">\n <img src=""./imag...","@@ -1,23 +1,25 @@\n <div class=""align-center"">...",add more info about nvidia gpus,"Pre-release 2023 December version (Mistral, Pr..."
1,4b97a810b509c93f44be4c037c7aa18fb8922884,"Pre-release 2023 December version (Mistral, Pr...",PNG\r\n\n,PNG\r\n\n,Binary files a/images/Discord.png and b/images...,distro binary files,"Pre-release 2023 December version (Mistral, Pr..."
2,4b97a810b509c93f44be4c037c7aa18fb8922884,"Pre-release 2023 December version (Mistral, Pr...",NaN,PNG\r\n\n,Binary files /dev/null and b/images/LAION 2GPU...,distinguish 2gpu image from null,"Pre-release 2023 December version (Mistral, Pr..."
3,4b97a810b509c93f44be4c037c7aa18fb8922884,"Pre-release 2023 December version (Mistral, Pr...","<?xml version=""1.0"" encoding=""utf-8"" standalon...",NaN,"@@ -1,1518 +0,0 @@\n-<?xml version=""1.0"" encod...",add missing missing nodes in skeleton skeleton,"Pre-release 2023 December version (Mistral, Pr..."
4,4b97a810b509c93f44be4c037c7aa18fb8922884,"Pre-release 2023 December version (Mistral, Pr...","<?xml version=""1.0"" encoding=""utf-8"" standalon...",NaN,"@@ -1,1424 +0,0 @@\n-<?xml version=""1.0"" encod...",add missing missing svg elements,"Pre-release 2023 December version (Mistral, Pr..."


In [16]:

import pandas as pd

analysis_df = pd.read_csv('Lab_2_analysis.csv')

# 1. Total number of commits and files
total_commits = analysis_df["Hash"].nunique()
total_files = analysis_df["Filename"].nunique()

# 2. Average number of modified files per commit
files_per_commit = analysis_df.groupby("Hash")["Filename"].nunique()
avg_files_per_commit = files_per_commit.mean()

fix_type_distribution = analysis_df["LLM_inference"].value_counts().reset_index()
fix_type_distribution.columns = ['Fix type','Count']
top_files = analysis_df["Filename"].value_counts().reset_index()
top_files.columns = ['Filename','Count']
analysis_df["Extension"] = analysis_df["Filename"].str.split(".").str[-1]
top_extensions = analysis_df["Extension"].value_counts().reset_index()
top_extensions.columns = ['Extension','Count']

# Display results
print("Total commits:", total_commits)
print("Total files:", total_files)
print("Average modified files per commit:", round(avg_files_per_commit,2))
print("\nFix type distribution:\n", fix_type_distribution)
print("\nMost frequently modified files:\n", top_files.head(10))
print("\nMost frequently modified extensions:\n", top_extensions.head(10))

Total commits: 346
Total files: 119
Average modified files per commit: 3.53

Fix type distribution:
                                 Fix type  Count
0                    add missing version     33
1                     add missing import     32
2                      update unsloth.py     18
3                 add missing docstrings     16
4                  add tests for unsloth     16
..                                   ...    ...
898    add tf.jit.approx_backward_kernel      1
899         add docs for _rope_embedding      1
900                   add libdevice.tanh      1
901  add missing log message for unsloth      1
902              update sample_params.rb      1

[903 rows x 2 columns]

Most frequently modified files:
              Filename  Count
0            llama.py    158
1           _utils.py    146
2         __init__.py     83
3           loader.py     78
4      pyproject.toml     76
5             save.py     71
6          mistral.py     51
7           mapper.py     48
8   

In [20]:
top_files.head()

,Filename,Count
0,llama.py,158
1,_utils.py,146
2,__init__.py,83
3,loader.py,78
4,pyproject.toml,76


In [17]:
top_extensions

,Extension,Count
0,py,1063
1,toml,76
2,md,73
3,png,14
4,txt,4
5,svg,2
6,gitignore,1
7,LICENSE,1
8,sh,1


$$
MI = \max \left[ 0,\ 100 \cdot \frac{171 - 5.2 \ln V - 0.23 G - 16.2 \ln L + 50 \sin(\sqrt{2.4 C})}{171} \right]
$$

In [13]:
df = fix_type_distribution.reset_index()
df.columns = ['Fix type','Count']
df

,Fix type,Count
0,add missing version,33
1,add missing import,32
2,update unsloth.py,18
3,add missing docstrings,16
4,add tests for unsloth,16
...,...,...
898,add tf.jit.approx_backward_kernel,1
899,add docs for _rope_embedding,1
900,add libdevice.tanh,1
901,add missing log message for unsloth,1


In [4]:
!pip install radon


   ---------------------------------------- 0/2 [mando]
   ---------------------------------------- 0/2 [mando]
   -------------------- ------------------- 1/2 [radon]
   -------------------- ------------------- 1/2 [radon]
   -------------------- ------------------- 1/2 [radon]
   -------------------- ------------------- 1/2 [radon]
   -------------------- ------------------- 1/2 [radon]
   ---------------------------------------- 2/2 [radon]



In [5]:
import pandas as pd
from radon.complexity import cc_visit
from radon.metrics import mi_visit
from radon.raw import analyze


def analyze_code(source_code: str):
    """Extract MI, CC, and LOC from given source code string."""
    try:
        # Maintainability Index
        mi_score = mi_visit(source_code, True)

        # Cyclomatic Complexity (sum across all blocks)
        cc_blocks = cc_visit(source_code)
        cc_score = sum(block.complexity for block in cc_blocks)

        # Lines of Code
        raw_metrics = analyze(source_code)
        loc = raw_metrics.loc

        return mi_score, cc_score, loc
    except Exception:
        return None, None, None


def process_dataframe(df: pd.DataFrame):
    """Run radon analysis on Source Code (before/current) for each row."""
    results = []

    for idx, row in df.iterrows():
        mi_before, cc_before, loc_before = analyze_code(str(row["Source Code (before)"]))
        mi_after, cc_after, loc_after = analyze_code(str(row["Source Code (current)"]))

        results.append({
            "MI_Before": mi_before,
            "MI_After": mi_after,
            "CC_Before": cc_before,
            "CC_After": cc_after,
            "LOC_Before": loc_before,
            "LOC_After": loc_after,
            "MI_Change": (mi_after - mi_before) if mi_before is not None and mi_after is not None else None,
            "CC_Change": (cc_after - cc_before) if cc_before is not None and cc_after is not None else None,
            "LOC_Change": (loc_after - loc_before) if loc_before is not None and loc_after is not None else None
        })

    # Merge with original dataframe
    return pd.concat([df, pd.DataFrame(results)], axis=1)


# Example usage
# df = pd.read_csv("your_dataset.csv")
df_processed = process_dataframe(analysis_df)

# Save results
df_processed.to_csv("radon_results.csv", index=False)
print(df_processed.head())

KeyboardInterrupt: 

In [ ]:
import pandas as pd
from radon.complexity import cc_visit
from radon.metrics import mi_visit
from radon.raw import analyze


def analyze_code(source_code: str):
    """Extract MI, CC, and LOC from given source code string."""
    try:
        # Maintainability Index
        mi_score = mi_visit(source_code, True)

        # Average Cyclomatic Complexity
        cc_blocks = cc_visit(source_code)
        cc_score =  sum(block.complexity for block in cc_blocks)/len(cc_blocks)

        # Lines of Code
        raw_metrics = analyze(source_code)
        loc = raw_metrics.loc

        return mi_score, cc_score, loc
    except Exception:
        return None, None, None


def process_dataframe(df: pd.DataFrame):
    """Run radon analysis on Source Code (before/current) for each row."""
    results = []

    for idx, row in df.iterrows():
        mi_before, cc_before, loc_before = analyze_code(str(row["Source Code (before)"]))
        mi_after, cc_after, loc_after = analyze_code(str(row["Source Code (current)"]))

        results.append({
            "MI_Change": (mi_after - mi_before) if mi_before is not None and mi_after is not None else None,
            "CC_Change": (cc_after - cc_before) if cc_before is not None and cc_after is not None else None,
            "LOC_Change": (loc_after - loc_before) if loc_before is not None and loc_after is not None else None
        })

    # Merging with original dataframe
    return pd.concat([df, pd.DataFrame(results)], axis=1)


df_processed = process_dataframe(final_df)
df_processed.to_csv("radon_results.csv", index=False)

In [1]:
!pip install sacrebleu

   ---------------------------------------- 0.0/4.0 MB ? eta -:--:--
   -- ------------------------------------- 0.3/4.0 MB ? eta -:--:--
   ---------- ----------------------------- 1.0/4.0 MB 2.4 MB/s eta 0:00:02
   --------------- ------------------------ 1.6/4.0 MB 2.9 MB/s eta 0:00:01
   ------------------ --------------------- 1.8/4.0 MB 2.1 MB/s eta 0:00:02
   ------------------------- -------------- 2.6/4.0 MB 2.4 MB/s eta 0:00:01
   ------------------------------- -------- 3.1/4.0 MB 2.4 MB/s eta 0:00:01
   -------------------------------------- - 3.9/4.0 MB 2.6 MB/s eta 0:00:01
   ---------------------------------------- 4.0/4.0 MB 2.6 MB/s  0:00:01

   ---------------------------------------- 0/4 [tabulate]
   ---------- ----------------------------- 1/4 [portalocker]
   ---------- ----------------------------- 1/4 [portalocker]
   -------------------- ------------------- 2/4 [lxml]
   -------------------- ------------------- 2/4 [lxml]
   -------------------- ---------------

In [ ]:
from sacrebleu.metrics import BLEU

# Initialize BLEU scorer
bleu = BLEU()

def token_similarity_bleu(before_code, after_code):
    sys = [after_code]
    refs = [[before_code]]
    
    score = bleu.corpus_score(sys, refs)
    return score.score / 100.0      # sacrebleu returns score out of 100, normalize to [0,1]

In [ ]:
results_df = df_processed.dropna(subset=["Source Code (before)", "Source Code (current)"])
results_df["Semantic_Similarity"] = results_df.progress_apply(
    lambda row: semantic_similarity(str(row["Source Code (before)"]), str(row["Source Code (current)"])), axis=1
)

results_df["Token_Similarity"] = results_df.progress_apply(
    lambda row: token_similarity_bleu(str(row["Source Code (before)"]), str(row["Source Code (current)"])), axis=1
)

In [ ]:
import numpy

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.metrics.pairwise import cosine_similarity

tokenizer = AutoTokenizer.from_pretrained("microsoft/codebert-base")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
model = AutoModel.from_pretrained("microsoft/codebert-base",device_map="auto")

def semantic_similarity(code1, code2):
    inputs = tokenizer([code1, code2], return_tensors="pt", padding=True, truncation=True)

    with torch.no_grad():
        outputs = model(**inputs)

    # Mean pooling over tokens
    embeddings = outputs.last_hidden_state.mean(dim=1)

    # Cosine similarity
    sim = cosine_similarity([embeddings[0].numpy()], [embeddings[1].numpy()])[0][0]
    return sim

In [ ]:
results_df = df_processed.dropna(subset=["Source Code (before)", "Source Code (current)"])
results_df["Semantic_Similarity"] = results_df.progress_apply(
    lambda row: semantic_similarity(str(row["Source Code (before)"]), str(row["Source Code (current)"])), axis=1
)

results_df["Token_Similarity"] = results_df.progress_apply(
    lambda row: token_similarity_bleu(str(row["Source Code (before)"]), str(row["Source Code (current)"])), axis=1
)

In [ ]:
def classify_semantic(sim):
    return "Minor" if sim >= 0.80 else "Major"

def classify_token(sim):
    return "Minor" if sim >= 0.75 else "Major"

results_df["Semantic_Class"] = results_df["Semantic_Similarity"].apply(classify_semantic)
results_df["Token_Class"] = results_df["Token_Similarity"].apply(classify_token)
results_df["Classes_Agree"] = results_df.apply(
    lambda row: "YES" if row["Semantic_Class"] == row["Token_Class"] else "NO", axis=1
)

In [21]:
import pandas as pd
pd.read_csv('radon_results.csv')

,Hash,Author,Message,Filename,Change Type,Source Code (before),Source Code (current),Diff,LLM_inference,Overall_diff,Rectified Message,MI_Before,MI_After,CC_Before,CC_After,LOC_Before,LOC_After,MI_Change,CC_Change,LOC_Change
0,4b97a810b509c93f44be4c037c7aa18fb8922884,Daniel Han,"Pre-release 2023 December version (Mistral, Pr...",README.md,MODIFY,"<div class=""align-center"">\n <img src=""./imag...","<div class=""align-center"">\n <img src=""./imag...","@@ -1,23 +1,25 @@\n <div class=""align-center"">...",add more info about nvidia gpus,"@@ -1,23 +1,25 @@\n <div class=""align-center"">...",add more info about nvidia gpus,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4b97a810b509c93f44be4c037c7aa18fb8922884,Daniel Han,"Pre-release 2023 December version (Mistral, Pr...",Discord.png,MODIFY,PNG\r\n\n,PNG\r\n\n,Binary files a/images/Discord.png and b/images...,distro binary files,"@@ -1,23 +1,25 @@\n <div class=""align-center"">...",add more info about nvidia gpus,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4b97a810b509c93f44be4c037c7aa18fb8922884,Daniel Han,"Pre-release 2023 December version (Mistral, Pr...",LAION 2GPU.png,ADD,NaN,PNG\r\n\n,Binary files /dev/null and b/images/LAION 2GPU...,distinguish 2gpu image from null,"@@ -1,23 +1,25 @@\n <div class=""align-center"">...",add more info about nvidia gpus,100.000000,NaN,0.0,NaN,1.0,NaN,NaN,NaN,NaN
3,4b97a810b509c93f44be4c037c7aa18fb8922884,Daniel Han,"Pre-release 2023 December version (Mistral, Pr...",LAION 2GPU.svg,DELETE,"<?xml version=""1.0"" encoding=""utf-8"" standalon...",NaN,"@@ -1,1518 +0,0 @@\n-<?xml version=""1.0"" encod...",add missing missing nodes in skeleton skeleton,"@@ -1,23 +1,25 @@\n <div class=""align-center"">...",add more info about nvidia gpus,NaN,100.000000,NaN,0.0,NaN,1.0,NaN,NaN,NaN
4,4b97a810b509c93f44be4c037c7aa18fb8922884,Daniel Han,"Pre-release 2023 December version (Mistral, Pr...",SlimOrca 1GPU.svg,DELETE,"<?xml version=""1.0"" encoding=""utf-8"" standalon...",NaN,"@@ -1,1424 +0,0 @@\n-<?xml version=""1.0"" encod...",add missing missing svg elements,"@@ -1,23 +1,25 @@\n <div class=""align-center"">...",add more info about nvidia gpus,NaN,100.000000,NaN,0.0,NaN,1.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1230,9050d6f5278e2373e66d17fc9ac8c2ec4f986e7e,Roland Tannous,tests for mxfp4 and quantized models merge fix...,run_test.sh,ADD,NaN,"#!/bin/bash\nset -e\n\necho ""=================...","@@ -0,0 +1,18 @@\n+#!/bin/bash\n+set -e\n+\n+e...",add missing line,"@@ -0,0 +1,18 @@\n+#!/bin/bash\n+set -e\n+\n+e...",add test for merged model,100.000000,NaN,0.0,NaN,1.0,NaN,NaN,NaN,NaN
1231,9050d6f5278e2373e66d17fc9ac8c2ec4f986e7e,Roland Tannous,tests for mxfp4 and quantized models merge fix...,test_merged_model.py,ADD,NaN,# inference_on_merged.py\nfrom unsloth import ...,"@@ -0,0 +1,55 @@\n+# inference_on_merged.py\n+...",add test for merge,"@@ -0,0 +1,18 @@\n+#!/bin/bash\n+set -e\n+\n+e...",add test for merged model,100.000000,79.770172,0.0,4.0,1.0,55.0,-20.229828,4.0,54.0
1232,9050d6f5278e2373e66d17fc9ac8c2ec4f986e7e,Roland Tannous,tests for mxfp4 and quantized models merge fix...,train_and_merge.py,ADD,NaN,# train_and_merge.py\nfrom unsloth import Fast...,"@@ -0,0 +1,71 @@\n+# train_and_merge.py\n+from...",add examples to train_and_merge,"@@ -0,0 +1,18 @@\n+#!/bin/bash\n+set -e\n+\n+e...",add test for merged model,100.000000,77.081798,0.0,6.0,1.0,71.0,-22.918202,6.0,70.0
1233,9050d6f5278e2373e66d17fc9ac8c2ec4f986e7e,Roland Tannous,tests for mxfp4 and quantized models merge fix...,test_merge_4bit_validation.py,ADD,NaN,from unsloth import FastLanguageModel\nfrom un...,"@@ -0,0 +1,223 @@\n+from unsloth import FastLa...",add test case for formatting prompts,"@@ -0,0 +1,18 @@\n+#!/bin/bash\n+set -e\n+\n+e...",add test for merged model,100.000000,54.395556,0.0,2.0,1.0,223.0,-45.604444,2.0,222.0
